In [1]:
%load_ext autoreload
%autoreload 2

from models.ranked_transformer import Moonshot
import torch, pytorch_lightning as pl

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from models.chemformer.tokeniser import MolEncTokeniser
from models.chemformer.utils import REGEX


path = "tempdata/chemformer/last.ckpt"
token_file = "tempdata/chemformer/bart_vocab.txt"

tokeniser = MolEncTokeniser.from_vocab_file(
  token_file, REGEX, 272
)
model = Moonshot.load_from_checkpoint(path, tokeniser=tokeniser).cuda()


Initialized SignCoordinateEncoder[512] with dims [224, 224, 64] and 2 positional encoders. 64 bits are reserved for encoding the final bit





Ranked Transformer callin super


In [9]:
from utils.init_utils.data_init import data_mux


feats = ["HSQC", "SMILES"]
feat_handlers = ["pad", "tokenise"]
ds_path = "tempdata/SMILES_dataset/"

gim = data_mux(feats, feat_handlers, ds_path, token_file = token_file)

In [10]:
gim.setup("fit")
val_dl = gim.val_dataloader()

In [11]:
for batch in val_dl:
  hsqc, collated_smiles = batch
  hsqc = hsqc.cuda()
  collated_smiles = {k:(v if type(v) is not torch.Tensor else v.cuda()) for k,v in collated_smiles.items()}
  break

In [16]:
out = model.sample_rdm(hsqc)

Max seq len: 512


100%|██████████| 511/511 [00:59<00:00,  8.60it/s]


In [19]:
raw = collated_smiles["raw_smiles"]

In [20]:
for r, o in zip(raw, out):
  print(r, o)

c12cc(O)cc3c1C(N(C)CC3)Cc1ccc(O)c(O)c1-2.Br ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
c12c(cc[nH]1)c(Br)cc(Br)c2 c1cccOCCOC(CNCC=CC=)()OC
c1c2[nH]c3cnccc3c2cc(Br)c1 Oc1((BrOC(Cn=2CCC)(OC=C))O)
c1c2cc[nH]c2ccc1Br Ccc))c)C)C=COCCCOCC)COCCCCCCC(C
C(#CC#CC=CCO)C#C Cc21C((CC=1CC(#C=#(CCC=CCCC()O(C
O=C1Cc2c(cccc2)C(C#CC#C)O1 O2C2O)C=(CCCN#CCO##)CCCCC##C(=
C(CCC=CCCCCCCCCCC=CC(C#C)=O)CCCCCCCCC(C#CC#CCO)O 422c))OCC1COCC)CCOC#CCCCC
C(C(C=CCCCCCCCCCCC=CC(C#C)=O)O)#C 4(C2cCCCCC1(cCCC=)CCCCCCCC#C(()(
BrC(C1CC(O)C(CC2C(Br)CC(Br)C(CC)O2)O1)C#C 6(2cBrCCCO)OCCC(COCC=)COCCCCCcCCC
C1(Br)C2(CCC(C(Br)CC)O2)OC2C1OC(C(Br)C#C)C2 OCCc()))CC)CCCCCC)C=#=(C(C=CCO=C
c1c2c(ccc1)cc(C(C#C)C)oc2=O c12cBrc(#CC=C1CCO=CCC(CC
C(CCCCC(=O)CCCCCC=CC(O)CCCCCCCCCCCCC(C(O)C(O)C#CC(O)=O)O)#CC(O)C#CC(O)C(CCCC(C=CC(O)C#C)O)O 2C
C(#CC(O)C(O)CCCC(C=CC(C#C)O)O)C(C#CCCCCC(CCCCCC=CC(CCCCCCCCCCCCCC(O)C(O)C#CC(O)=O)O)=O)O 1(Ccc)(C)C
C(C#C)(C=CC(CCCCCCCCCCCC=CCCCCCCCCC(O)C=CC(C#C)O)O)O cCc[nH]c
C(CCCC(O)C=CC(C#C)O)CCCCCCCCC=CCCCCCCCCC(O)C=CC(C#C)O